# L46 Mini Project

## Imports and device setup

In [40]:
import time
from typing import Optional, Tuple, Literal
import json
import math
import types

import torch
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    PreTrainedTokenizerBase,
    PreTrainedModel,
)

from torch import nn
import torch.utils.checkpoint

from transformers.models.llama.modeling_llama import (
    LlamaAttention,
    rotate_half,
    repeat_kv,
)

from transformers.cache_utils import Cache

In [41]:
# change appropriately
device = "mps"

## Preliminaries

### Chosen models

GPT2 and Llama3.2-1B.

TODO: 
- [ ] Write about them and their specifics. (PEs, attention, dimensions, num parameters)
- [ ] Show example generations.

In [42]:
def load_model_tokenizer(
    model_path: str,
    output_attn: bool = False,
    attn_implementation: Optional[str] = None,
) -> tuple[PreTrainedModel, PreTrainedTokenizerBase]:
    # TODO add docstring
    model = AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_path,
        output_attentions=output_attn,
        attn_implementation=attn_implementation,
    ).to(device)

    tokenizer = AutoTokenizer.from_pretrained(model_path)

    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id

    return model, tokenizer

In [4]:
llama3_model_path = "/Users/aszab/repos/models/Llama-3.2-1B"
llama3_model, llama3_tokenizer = load_model_tokenizer(
    llama3_model_path, attn_implementation="eager"
)

In [5]:
gpt2_model_path = "/Users/aszab/repos/models/gpt2"
gpt2_model, gpt2_tokenizer = load_model_tokenizer(
    gpt2_model_path, attn_implementation="eager"
)

In [6]:
def generate_text(
    prompt: str,
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizerBase,
    num_tokens: int = 10,
) -> list[str]:
    # TODO add docstring
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generations = []

    for _ in range(num_tokens):
        with torch.no_grad():
            outputs = model(
                **inputs,
            )
        pred_token_idx = outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)
        inputs["input_ids"] = torch.cat([inputs["input_ids"], pred_token_idx], dim=-1)
        inputs["attention_mask"] = torch.cat(
            [inputs["attention_mask"], torch.ones_like(pred_token_idx)], dim=-1
        )
        pred_token = tokenizer.decode(pred_token_idx[0])
        generations.append(pred_token)

    return generations

GPT-2 example generation

In [ ]:
start_time = time.time()
prompt = "Hello, my dog is cute and"
print("".join(generate_text(prompt, gpt2_model, gpt2_tokenizer, 100)))
print(f"\n---\n{time.time() - start_time:.2f} seconds")

 I love him. I'm not sure if he's a good dog, but I'm sure he's a good dog. I'm not sure if he's a good dog, but I'm sure he's a good dog.

I'm

---
1.89 seconds


LLama3.2-1B generation

In [35]:
start_time = time.time()
prompt = "Hello, my dog is cute and"
print("".join(generate_text(prompt, llama3_model, llama3_tokenizer, 100)))
print(f"\n---\n{time.time() - start_time:.2f} seconds")

 playful, but he is very aggressive and he bites me when I try to pet him. I have tried to train him, but he is very stubborn and he doesn’t listen to me. I have tried to use a leash, but he still bites

---
8.06 seconds


#TODO: Explain why attention scales quadratically. (Maybe show one of the graphs from the website? https://medium.com/@joaolages/kv-caching-explained-276520203249)

Instead of calculating the attention components at each decoding step, we can reuse the already computed keys and values by caching them and using in the next decoding step.

Conveniently, the default implementation of both GPT2 and Llama3.2 from the `transformers` library returns the computed keys and values during a generation in the `outputs.past_key_values` object of type `tuple[tuple[torch.Tensor[batch_size, attn_head_num, seq_num, attn_head_dim]]]`, where the first tuple is of the dimension of the number of decoder layers, while the second is 2-dimensional with the first element being keys and the latter being values.


In [36]:
def generate_text_kv_caching(
    prompt: str,
    model: PreTrainedModel,
    tokenizer: PreTrainedTokenizerBase,
    num_tokens: int = 10,
    kv_cache_size: int = -1,
) -> list[str]:
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    generations = []

    # First compute all the KV caches for the prompt
    with torch.no_grad():
        outputs = model(
            **inputs,
        )
    past_key_values = outputs.past_key_values
    generations.append(
        tokenizer.decode(outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)[0])
    )

    # Now generate the rest of the tokens one by one
    for _ in range(num_tokens):
        pred_token_idx = outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)
        with torch.no_grad():
            outputs = model(
                pred_token_idx,
                labels=None,
                past_key_values=past_key_values,
                use_cache=True,
            )
        past_key_values = outputs.past_key_values
        if kv_cache_size != -1 and past_key_values[0][0].shape[2] >= kv_cache_size:
            past_key_values = [
                [param[:, :, -kv_cache_size:, :] for param in layer]
                for layer in past_key_values
            ]

        generations.append(
            tokenizer.decode(outputs.logits[:, -1, :].argmax(dim=-1).unsqueeze(1)[0])
        )
    return generations

In [ ]:
start_time = time.time()
prompt = "Hello, my dog is cute and"
print(
    "".join(
        generate_text_kv_caching(
            prompt,
            gpt2_model,
            gpt2_tokenizer,
            100,
        )
    )
)
print(f"\n---\n{time.time() - start_time:.2f} seconds")

In [37]:
start_time = time.time()
prompt = "Hello, my dog is cute and"
print(
    "".join(
        generate_text_kv_caching(
            prompt,
            llama3_model,
            llama3_tokenizer,
            100,
        )
    )
)
print(f"\n---\n{time.time() - start_time:.2f} seconds")

 playful, but he is very aggressive and he bites me when I try to pet him. I have tried to train him, but he is very stubborn and he doesn’t listen to me. I have tried to use a leash, but he still bites me

---
2.59 seconds


When the cache capacity is exceeded, some of the tokens need to be evicted. The default cache implementation, also known as *window attention* uses the least recently used (LRU) cache eviction policy. Let's set the `kv_cache_size`=20 tokens and observe the results. 

In [ ]:
start_time = time.time()
prompt = "Hello, my dog is cute and"
print(
    "".join(
        generate_text_kv_caching(
            prompt,
            llama3_model,
            llama3_tokenizer,
            50,
            kv_cache_size=20,
        )
    )
)
print(f"\n---\n{time.time() - start_time:.2f} seconds")

 playful, but he is very aggressive and he bites me when I try, he dog bites me bites me aggressive aggressive aggressive aggressive aggressive aggressive aggressive bites bites bites bites bites bites bites biteseseseseseseseseseseseseseseses

---
2.21 seconds


As expected from the original paper by Xiao et al., the model's ability to generate text drastically vanishes. The authors argue that it is caused by ejecting the position from the kv cache, which in fact serves as an attention sink -- a place to "dump" attention score resulting from its softmax formulation. 

#TODO Describe more why this happens

They propose a novel method, called sink attention, which modifies the cache eviction policy by always keeping the first *n* tokens.

#TODO add attention diagrams.

## Experiment 1

The first experiment serves as a way to replicate and deepen the results obtained by Xiao et al., using a new generation model -- Llama3.2-1B. 

Following Xiao et al., we implement 4 types of attention computations:
- Dense attention
- Window attention
- Sliding window with recomputation
- Sink attention

We compare the 4 implementation using a commonly used test split of the `wikitext-2` dataset to evaluate model perplexity, generation time and kv-cache memory usage of GPT-2 and LLama3.2.

### Implementation of sink attention forward pass 

However, implementing sink attention additionally involves shifting the positional encodings, which are added to the keys and values for attention computation. This allows the model to decode sequences even past the maximum sequence length on which it was trained.

Most modern models use Rotary Positional Embeddings (RoPE), relative positional encodings or ALiBi (Attention with Linear Biases), which directly affect how attention is computed. Llama3.2 uses RoPEs, which are added to the keys and queries before they are stored in the kv cache. Since sink attention involves shifting positions of stored keys and values, it is necessary to reimplement the attention pass for the model. To do so, I will modify the [original `transformers` implementation of the *eager attention* forward pass](https://github.com/huggingface/transformers/blob/241c04d36867259cdf11dbb4e9d9a60f9cb65ebc/src/transformers/models/llama/modeling_llama.py#L290-L358) and replace all its calls with the modified version.

#TODO explain that the positional encodings need to be recomputed each time the attention pass is performed but some computation can be saved if implemented from scratch.

In [43]:
def apply_rotary_pos_emb_x(x, cos, sin, unsqueeze_dim=1):
    cos = cos.unsqueeze(unsqueeze_dim)
    sin = sin.unsqueeze(unsqueeze_dim)
    x_embed = (x * cos) + (rotate_half(x) * sin)
    return x_embed


def llama_modified_attention_forward(
    self,
    hidden_states: torch.Tensor,
    attention_mask: Optional[torch.Tensor] = None,
    past_key_value: Optional[Cache] = None,
    output_attentions: bool = False,
    position_embeddings: Optional[Tuple[torch.Tensor, torch.Tensor]] = None,
    **kwargs,
) -> Tuple[torch.Tensor, Optional[torch.Tensor], Optional[Tuple[torch.Tensor]]]:
    bsz, q_len, _ = hidden_states.size()

    query_states = self.q_proj(hidden_states)
    key_states = self.k_proj(hidden_states)
    value_states = self.v_proj(hidden_states)

    # use -1 to infer num_heads and num_key_value_heads as they may vary if tensor parallel is used
    query_states = query_states.view(bsz, q_len, -1, self.head_dim).transpose(1, 2)
    key_states = key_states.view(bsz, q_len, -1, self.head_dim).transpose(1, 2)
    value_states = value_states.view(bsz, q_len, -1, self.head_dim).transpose(1, 2)

    # Figure out the length of the key value sequence given current and cached key value states
    kv_seq_len = key_states.shape[-2]
    if past_key_value is not None and len(past_key_value) > self.layer_idx:
        kv_seq_len += past_key_value[self.layer_idx][0].shape[-2]

    # If we have a cache, we can use it to avoid recomputing the key value states.
    # NOTE: Keys are stored without positional encodings added.
    if past_key_value is not None:
        key_states, value_states = past_key_value.update(
            key_states,
            value_states,
            self.layer_idx,
        )

    # Recompute rotary_emb for the entire sequence. In theory this could be done once when the model is instantiated.
    cos, sin = self.rotary_emb(
        value_states,
        position_ids=torch.arange(kv_seq_len, device=value_states.device).unsqueeze(0),
    )

    # Compute keys and queries with rotary embeddings separately given they may have different lengths due to caching.
    key_states = apply_rotary_pos_emb_x(key_states, cos, sin)
    query_states = apply_rotary_pos_emb_x(
        query_states, cos[:, -q_len:, :], sin[:, -q_len:, :]
    )

    # Continue with the original transformers Llama implementation
    key_states = repeat_kv(
        key_states,
        self.num_key_value_groups,
    )
    value_states = repeat_kv(
        value_states,
        self.num_key_value_groups,
    )
    attn_weights = torch.matmul(
        query_states,
        key_states.transpose(2, 3),
    ) / math.sqrt(self.head_dim)

    if attention_mask is not None:  # no matter the length, we just slice it
        causal_mask = attention_mask[:, :, :, : key_states.shape[-2]]
        attn_weights = attn_weights + causal_mask

    # upcast attention to fp32
    attn_weights = nn.functional.softmax(
        attn_weights,
        dim=-1,
        dtype=torch.float32,
    ).to(query_states.dtype)
    attn_weights = nn.functional.dropout(
        attn_weights, p=self.attention_dropout, training=self.training
    )
    attn_output = torch.matmul(attn_weights, value_states)

    if attn_output.size() != (bsz, self.num_heads, q_len, self.head_dim):
        raise ValueError(
            f"`attn_output` should be of size {(bsz, self.num_heads, q_len, self.head_dim)}, but is"
            f" {attn_output.size()}"
        )

    attn_output = attn_output.transpose(1, 2).contiguous()

    attn_output = attn_output.reshape(bsz, q_len, -1)

    attn_output = self.o_proj(attn_output)

    if not output_attentions:
        attn_weights = None

    return attn_output, attn_weights, past_key_value


# Replace all default llama attention modules with the new implementation above.
def enable_llama_pos_shift_attention(model):
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            enable_llama_pos_shift_attention(
                module,
            )

        if isinstance(module, LlamaAttention):
            model._modules[name].forward = types.MethodType(
                llama_modified_attention_forward, model._modules[name]
            )

#### Verify the correctness of the implementation

Here we verify the correctness of the attention forward pass implementation.

#TODO explain how we are verifying the correctness 

In [46]:
def load_data():
    data_wikitext = load_dataset(
        "wikitext", "wikitext-2-raw-v1", split="test", cache_dir="~/repos/datasets"
    )
    data_wikitext = "\n\n".join(
        list(filter(lambda x: len(x) > 0, data_wikitext["text"]))
    )

    return data_wikitext

In [47]:
data_wikitext = load_data()

encodings = llama3_tokenizer(data_wikitext, return_tensors="pt")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Token indices sequence length is longer than the specified maximum sequence length for this model (288937 > 131072). Running this sequence through the model will result in indexing errors


In [44]:
llama3_model_path = "/Users/aszab/repos/models/Llama-3.2-1B"
llama3_model, llama3_tokenizer = load_model_tokenizer(
    llama3_model_path, attn_implementation="eager"
)

In [45]:
enable_llama_pos_shift_attention(llama3_model)

In [48]:
reference_llama3_model, _ = load_model_tokenizer(
    llama3_model_path, attn_implementation="eager"
)

#### Qualitative evaluation on a sample text

In [51]:
prompt = "Hello, my dog is cute and"
num_tokens = 17
generation = generate_text_kv_caching(
    prompt,
    llama3_model,
    llama3_tokenizer,
    num_tokens=num_tokens,
)
reference_generation = generate_text_kv_caching(
    prompt,
    reference_llama3_model,
    llama3_tokenizer,
    num_tokens=num_tokens,
)

print(prompt + "...\n")
print("modified model | reference model")
for gen, ref_gen in zip(generation, reference_generation):
    print(gen + (len("modified model") - len(gen)) * " ", ref_gen, sep=" |")

Hello, my dog is cute and...

modified model | reference model
 playful       | playful
,              |,
 but           | but
 he            | he
 is            | is
 very          | very
 aggressive    | aggressive
 and           | and
 he            | he
 bites         | bites
 me            | me
 when          | when
 I             | I
 try           | try
 to            | to
 pet           | pet
 him           | him
.              |.


#### Test logits given a longer sequence

In [52]:
print(llama3_tokenizer.decode(encodings.input_ids[:, :512][0]))

<|begin_of_text|> = Robert Boulter = 


 Robert Boulter is an English film, television and theatre actor. He had a guest @-@ starring role on the television series The Bill in 2000. This was followed by a starring role in the play Herons written by Simon Stephens, which was performed in 2001 at the Royal Court Theatre. He had a guest role in the television series Judge John Deed in 2002. In 2004 Boulter landed a role as " Craig " in the episode " Teddy's Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi. He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur, which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London. He was directed by John Tiffany and starred alongside Ben Whishaw, Shane Zaza, Harry Kent, Fraser Ayres, Sophie Stanton and Dominic Hall. 


 In 2006, Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill. He appeared on a 20

In [56]:
with torch.no_grad():
    outputs = llama3_model(
        encodings.input_ids[:, :512].to(device),
    )
    outputs_final = llama3_model(
        encodings.input_ids[:, :512].to(device),
        past_key_values=outputs.past_key_values,
    )

    reference_outputs = reference_llama3_model(
        encodings.input_ids[:, :512].to(device),
    )
    reference_outputs_final = reference_llama3_model(
        encodings.input_ids[:, :512].to(device),
        past_key_values=reference_outputs.past_key_values,
    )

In [57]:
assert torch.equal(outputs.logits, reference_outputs.logits)
assert torch.equal(outputs_final.logits, reference_outputs_final.logits)

#### Test caching across multiple generations

In [ ]:
past_key_values = None
reference_past_key_values = None

for idx in tqdm(range(1024)):
    tokens = encodings.input_ids[:, idx : idx + 1].to(device)
    with torch.no_grad():
        outputs = llama3_model(
            tokens,
            past_key_values=past_key_values,
        )
        past_key_values = outputs.past_key_values

        reference_outputs = reference_llama3_model(
            tokens,
            past_key_values=reference_past_key_values,
        )
        reference_past_key_values = reference_outputs.past_key_values

    assert torch.equal(outputs.logits, reference_outputs.logits)

    # clean gpu cache on apple silicon
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

## Experiment 2

## Experiment 3